In [1]:
from google.colab import drive
drive.mount('/content/drive')

%tensorflow_version 1.x 

import urllib.request
import pandas as pd
import numpy as np

import tensorflow_hub as hub
import tensorflow as tf

import tensorflow.keras
from tensorflow import keras
from tensorflow.keras import backend as K

from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense, Input
from keras import optimizers
from tensorflow.keras.models import Model,Sequential

import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
import nltk
nltk.download('punkt')

Mounted at /content/drive
TensorFlow 1.x selected.


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def split(data):
  #level이 1인 index list
  index1=[]
  for i in range(len(data)) :
    if data['level'][i]==1:
      index1.append(i)

  #level이 2인 index list
  index2=[]
  for i in range(len(data)) :
    if data['level'][i]==2:
      index2.append(i)
  '''
  #level이 3인 index list
  index3=[]
  for i in range(len(data)) :
    if data['level'][i]==3:
      index3.append(i)
  '''
  print("level 1 개수 : ",len(index1))
  print("level 2 개수 : ",len(index2))  
  #print("level 3 개수 : ",len(index3))
  
  import random  #random.sample(list,k) -> list에서 k개 출력

  datalen=len(data)
  trainlen=int(datalen*0.8)
  testlen=datalen-trainlen

  index_test=random.sample(index1,int(testlen*0.3))
  index_test.extend(random.sample(index2,int(testlen*0.3)))
  #index_test.extend(random.sample(index3,int(testlen*0.3)))
  index_test.sort()
  print("test 개수 : ",len(index_test))

  index_train=[]
  for i in range(0,datalen):        
      if i not in index_test:
          index_train.append(i)
  print("train 개수 : ",len(index_train))

  #train index와 test index가 겹치지 않는 것을 확인
  for i in index_train:        
      if i in index_test:
          print("!!!")

  return index_train, index_test

**1. 데이터 불러오기**

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/인문감정/w2v_softmax')
%pwd

raw_data= pd.read_csv('감정문장_1207_level.csv' , header=0)
dataset=raw_data.replace('n',0).replace('y',1)
dataset=dataset.replace('수심',1).replace('비탄',2).replace('우울',3)
dataset.head(3)

,Unnamed: 0,sentence,korean,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,level
0,0,"""Lawrence Selden went from here to find you.""\...",“로렌스 셀던이 여기 있다가 너를 찾으러 갔어.”\n이 말에 돌처럼 단단하게 굳어 ...,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2
1,1,"Depression?is?often?triggered?by?loss,?not?onl...","우울증을 유발하는 계기는 상실일 때가 많다. 잃은 것은 사람일 수도 있지만, 우리가...",1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2
2,2,The next day was a dreary one for Emma. Everyt...,엠마에게 다음 날은 우울한 하루였다. 모든 것이 사물의 표면에 어지럽게 떠다니는 검...,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2


In [ ]:
del dataset['Unnamed: 0']
dataset.head(3)

,sentence,korean,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,level
0,"""Lawrence Selden went from here to find you.""\...",“로렌스 셀던이 여기 있다가 너를 찾으러 갔어.”\n이 말에 돌처럼 단단하게 굳어 ...,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2
1,"Depression?is?often?triggered?by?loss,?not?onl...","우울증을 유발하는 계기는 상실일 때가 많다. 잃은 것은 사람일 수도 있지만, 우리가...",1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2
2,The next day was a dreary one for Emma. Everyt...,엠마에게 다음 날은 우울한 하루였다. 모든 것이 사물의 표면에 어지럽게 떠다니는 검...,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2


**2. 전처리 과정**

> 1) train, test index 분리
>>*사용자 정의 함수( split() ) 사용*

> 2) 문장 전처리

In [ ]:
index_train, index_test = split(dataset)

X_train=dataset['sentence'][index_train]
X_test=dataset['sentence'][index_test]

X_train=np.array(X_train)
X_test=np.array(X_test)

level 1 개수 :  21
level 2 개수 :  62
test 개수 :  10
train 개수 :  73


In [ ]:
# 특수 문자 제거
dataset['sentence'] = dataset['sentence'].str.replace("[^a-zA-Z]", " ")
# 전체 단어에 대한 소문자 변환
dataset['sentence'] = dataset['sentence'].apply(lambda x: x.lower())

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

review_lines=list()
lines=dataset['sentence'].values.tolist()

for line in lines:
  tokens=word_tokenize(line) #영어 corpus 토큰화
  tokens=[w.lower() for w in tokens]
  table=str.maketrans('','',string.punctuation)
  stripped=[w.translate(table) for w in tokens]
  words=[word for word in stripped if word.isalpha()]
  stop_words=set(stopwords.words("english"))
  words=[w for w in words if not w in stop_words]
  review_lines.append(words)
  
print(len(review_lines))

83


In [ ]:
'''
#문자열을 인덱스의 리스트로 변환
X_train_tokens=tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens=tokenizer_obj.texts_to_sequences(X_test)

X_train_pad= pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding='post')
'''

"\n#문자열을 인덱스의 리스트로 변환\nX_train_tokens=tokenizer_obj.texts_to_sequences(X_train)\nX_test_tokens=tokenizer_obj.texts_to_sequences(X_test)\n\nX_train_pad= pad_sequences(X_train_tokens, maxlen=max_length, padding='post')\nX_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding='post')\n"

**3. word2vec 임베딩**

In [ ]:
import gensim
EMBEDDING_DIM = 100

model=gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words=list(model.wv.vocab)
print(len(words))

filename='1207_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename,binary=False)

import os

embeddings_index={}
f=open(os.path.join('','1207_embedding_word2vec.txt'),encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:])
  embeddings_index[word]=coefs
f.close()

1745


**4. 패딩(Padding)**

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences=tokenizer_obj.texts_to_sequences(review_lines)

word_index=tokenizer_obj.word_index

max_length=max(len(s.split()) for s in dataset['sentence'])
max_length
vocab_size=len(tokenizer_obj.word_index)+1
vocab_size

review_pad=pad_sequences(sequences,maxlen=max_length)

X_train_pad=review_pad[index_train]
X_test_pad=review_pad[index_test]

num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,EMBEDDING_DIM))

for word, i in word_index.items():
  if i >num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None :
    embedding_matrix[i]=embedding_vector

**5. softmax를 이용하여 예측**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras import optimizers

model=Sequential()
embedding_layer=Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), input_length=max_length, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
adam = optimizers.Adam(lr=0.01,decay=0.1, amsgrad=False)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 366, 100)          174600    
_________________________________________________________________
flatten_10 (Flatten)         (None, 36600)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 73202     
Total params: 247,802
Trainable params: 73,202
Non-trainable params: 174,600
_________________________________________________________________


In [ ]:
#y1~y28까지 반복해서 예측
total_pred=[]
acc=[]
for i in range(2,30):
      print("<<",dataset.columns[i],">>")
      y_train=np.asarray(dataset.iloc[index_train,i])
      y_test=np.asarray(dataset.iloc[index_test,i])
      y_train2=keras.utils.to_categorical(y_train,num_classes=2)
      y_test2=keras.utils.to_categorical(y_test,num_classes=2)

      print("** train **")
      history=model.fit(X_train_pad, y_train2, epochs=5, batch_size=34)
      print("** test **")
      e_history=model.evaluate(X_test_pad, y_test2, batch_size=34)
      print("** predict **")
      xhat = X_test_pad
      pred = model.predict(xhat)

      prediction=[]
      for i in range(len(X_test_pad)) :
        if pred[i,0]>pred[i,1]:
            max_position=0
        else:
          max_position=1
        prediction.append(max_position)  
      
      total_pred.append(prediction)
      acc.extend(history.history['accuracy'])

<< y1 >>
** train **
Epoch 1/5
73/73 [==============================] - 0s 2ms/step - loss: 0.6857 - accuracy: 0.7534
Epoch 2/5
73/73 [==============================] - 0s 330us/step - loss: 0.6220 - accuracy: 1.0000
Epoch 3/5
73/73 [==============================] - 0s 407us/step - loss: 0.5842 - accuracy: 1.0000
Epoch 4/5
73/73 [==============================] - 0s 341us/step - loss: 0.5559 - accuracy: 1.0000
Epoch 5/5
73/73 [==============================] - 0s 397us/step - loss: 0.5329 - accuracy: 1.0000
** test **
10/10 [==============================] - 0s 7ms/step
** predict **
<< y2 >>
** train **
Epoch 1/5
73/73 [==============================] - 0s 326us/step - loss: 0.6243 - accuracy: 0.6849
Epoch 2/5
73/73 [==============================] - 0s 336us/step - loss: 0.6192 - accuracy: 0.6849
Epoch 3/5
73/73 [==============================] - 0s 377us/step - loss: 0.6139 - accuracy: 0.6849
Epoch 4/5
73/73 [==============================] - 0s 355us/step - loss: 0.6083 - accuracy

In [ ]:
import numpy as np
np.mean(acc)

0.344227

In [ ]:
total_pred

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
true_y=[]
for i in range(2,30):
  true_y.append(dataset.iloc[index_test,i].tolist())

true_y

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 1, 0, 0, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
false_num=0
for pred_list, true_list in zip(total_pred,true_y):
  for i in range(0,len(index_test)):
    if pred_list[i] != true_list[i]:
      false_num+=1

In [ ]:
acc = 1-(false_num/(len(index_test)*28))
print("예측 정확도 :",acc)

예측 정확도 : 0.40714285714285714
